### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

### Описание данных

* train.csv - данные для обучения
* test.csv - данные для подготовки самбита и проверки
* sampleSubmission.csv - пример корректного но бесполезного сабмита
* other.csv - необязательные данные для доп.статистик и прочих извращений (например обучение word2vec-а)

### Описание полей

* id - внутренний идетификатор
* name - название вакансии
* description - текст вакансии
* target - класс заинтересованности

In [1]:
%%time

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, \
                        accuracy_score, classification_report, roc_auc_score, roc_curve

CPU times: user 443 ms, sys: 159 ms, total: 602 ms
Wall time: 880 ms


## Шаг 1. Импортируем все в Pandas


In [2]:
%%time
sample_submission = pd.read_csv('sampleSubmission.csv',',')
train_df = pd.read_csv('train.csv','\t')
test_df = pd.read_csv('test.csv','\t')
other_df = pd.read_csv('other.csv','\t')

CPU times: user 18.6 s, sys: 2.39 s, total: 21 s
Wall time: 21.1 s


In [3]:
train_df.shape

(200000, 4)

In [4]:
train_df.columns.values

array(['id', 'name', 'description', 'target'], dtype=object)

In [5]:
print(train_df['description'][0])

<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на постоянную работу руководителя секции (хозтовары).</strong></p> <p><strong>Обязанности: </strong></p> <p> </p> <ul> <li>организация эффективного и культурного обслуживания покупателей</li> <li>организация приемки-сдачи товаров на склад</li> <li>консультация покупателей по вопросам, касающимся оказываемых услуг</li> <li>контроль своевременной подачи товаров в торговую секцию</li> <li>проверка качества, сроков годности ТМЦ, наличие маркировок, ценников на товарах</li> <li>организация бесперебойной работы товарной секции и участка склада, закрепленного за ней</li> <li>контроль за сохранностью товаров, торгового оборудования и прочих материальных ценностей</li> <li>инвентаризация товаров</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>знание товарной группы</li> <li>знание правил приемки товара</li> <li>опыт работы в розничной торговле в качестве продавца 6 р. / заместителя заведующего секцией/отделом</li> <li>ответ

## Шаг 2. Чистим данные и подготавливаем текст

In [6]:
# Подключаем БьютифулСуп
from bs4 import BeautifulSoup             

# Прогоняем отдельное описание через БьютифулСуп   
example1 = BeautifulSoup(train_df['description'][0], 'html5lib')  

In [7]:
#Подключаем регулярки, чтобы очистить текст от тегов html
import re
# Делаем найти и заменить на ничто для всего что не похоже на регулярку
letters_only = re.sub("[^а-яА-Яa-zA-Z]",           # находим всё что не
                      " ",                   # Меняем на пустоту
                      example1.get_text() )  
print (letters_only)

В НОВЫЙ МАГАЗИН YORK  хозтовары  приглашаем на постоянную работу руководителя секции  хозтовары   Обязанности      организация эффективного и культурного обслуживания покупателей организация приемки сдачи товаров на склад консультация покупателей по вопросам  касающимся оказываемых услуг контроль своевременной подачи товаров в торговую секцию проверка качества  сроков годности ТМЦ  наличие маркировок  ценников на товарах организация бесперебойной работы товарной секции и участка склада  закрепленного за ней контроль за сохранностью товаров  торгового оборудования и прочих материальных ценностей инвентаризация товаров    Требования   знание товарной группы знание правил приемки товара опыт работы в розничной торговле в качестве продавца   р    заместителя заведующего секцией отделом ответственность  лидерские качества  опыт управления коллективом      Условия   НОВЫЙ современный магазин розничной торговли хозтоварами хороший коллектив достойная зарплата  премии  соц пакет график работы 

In [8]:
lower_case = letters_only.lower()
words = lower_case.split()  

In [9]:
import nltk

#nltk.download()  # Если NLTK не установлен, то надо раскомментровать этот блок и исполнить

In [10]:
from nltk.corpus import stopwords # Импортируем стоп-слова русскго языка
print( stopwords.words('russian') )

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [11]:
# Удаляем стоп-слова русского языка из текста words
words = [w for w in words if not w in stopwords.words('russian')]
print (words)
# Посмоторим что получится, если прогнать стеммером. 
# Но вариант со стеммером будет в следующей версии домашней работы
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer( 'russian' )
words = [snowball_stemmer.stem(w) for w in words if not w in stopwords.words('russian')]
print (words)

['новый', 'магазин', 'york', 'хозтовары', 'приглашаем', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары', 'обязанности', 'организация', 'эффективного', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'склад', 'консультация', 'покупателей', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'участка', 'склада', 'закрепленного', 'контроль', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'розничной', 'торговле', 'качестве', 'продавца', 'р', 'заместителя', 'заведующего', 'секцией', 'отделом', 'ответственность', 'лидерские',

In [12]:
def vac_to_words( description, Stemmer ):
    """
    Функция делает преобразование поля description из формата html в формат с отдельными значащими словами
    На вход передается  описание и параметр True/False. Если True, то работает стеммер, иначе нет
    """
    description_text = BeautifulSoup(description, 'html5lib').get_text() #получаем текст      
    letters_only = re.sub('[^а-яА-Яa-zA-Z]', ' ', description_text) #удаляем всё что не буквы
    words = letters_only.lower().split() #получаем слова в нижнем регистре                             
    stops = set(stopwords.words('russian'))  # формируем set стоп слов русского языка                  
    if Stemmer: #удаляем минус фразы и  оставляем либо "стеммы", либо значащие слова
        meaningful_words = [snowball_stemmer.stem(w) for w in words if not w in stops]
    else: 
        meaningful_words = [w for w in words if not w in stops]
    return( ' '.join( meaningful_words ))  # собираем слова в строку с разделителеим пробел 

In [13]:
clean_vac = vac_to_words( train_df['description'][0] , True)
print(clean_vac)

нов магазин york хозтовар приглаша постоя работ руководител секц хозтовар обязан организац эффективн культурн обслуживан покупател организац приемк сдач товар склад консультац покупател вопрос каса оказыва услуг контрол своевремен подач товар торгов секц проверк качеств срок годност тмц налич маркировок ценник товар организац бесперебойн работ товарн секц участк склад закреплен контрол сохран товар торгов оборудован проч материальн ценност инвентаризац товар требован знан товарн групп знан прав приемк товар оп работ розничн торговл качеств продавц р заместител завед секц отдел ответствен лидерск качеств оп управлен коллектив услов нов современ магазин розничн торговл хозтовар хорош коллект достойн зарплат прем соц пакет график работ пн пт плава мест работ ул никифоров уруч


In [14]:
# Посмотрим сколько всего описаний есть в train_df
num_vacs = train_df['description'].size
num_vacs

200000

In [15]:
%%time
clean_train_vacs = [] #делаем пустой лист для записи всех описаний с отдельными словами

print ('Очищаем и собираем нормализованные описания из train_df...\n')

# Пройдем в цикле по всем описаниям
for i in range( 0, num_vacs ):
    
    # Для каждого описания вызываем функцию нормировки текста описания и добавляем нормированный текст в лист
    
    if( (i+1)%10000 == 0 ): #чтобы не было скучно, каждые 10 000 описаний будем печатать развлекательный текст
        print ('Описание {} из {} в обработке...'.format( i+1, num_vacs)) 
        
    clean_train_vacs.append( vac_to_words( train_df['description'][i] , True) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из train_df...

Описание 10000 из 200000 в обработке...
Описание 20000 из 200000 в обработке...
Описание 30000 из 200000 в обработке...
Описание 40000 из 200000 в обработке...
Описание 50000 из 200000 в обработке...
Описание 60000 из 200000 в обработке...
Описание 70000 из 200000 в обработке...
Описание 80000 из 200000 в обработке...
Описание 90000 из 200000 в обработке...
Описание 100000 из 200000 в обработке...
Описание 110000 из 200000 в обработке...
Описание 120000 из 200000 в обработке...
Описание 130000 из 200000 в обработке...
Описание 140000 из 200000 в обработке...
Описание 150000 из 200000 в обработке...
Описание 160000 из 200000 в обработке...
Описание 170000 из 200000 в обработке...
Описание 180000 из 200000 в обработке...
Описание 190000 из 200000 в обработке...
Описание 200000 из 200000 в обработке...
Я закончила!!!
CPU times: user 27min 42s, sys: 11.5 s, total: 27min 53s
Wall time: 27min 58s


## Создаём свойства через Bag of Words

In [16]:
%%time
print ('Фигачим Bag of words...\n')
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 


# 5 000 фич считались 20.1 сек
# прогоняем полученный в прошлом пункте лист с текстами через fit_transform() 
train_data_features = vectorizer.fit_transform(clean_train_vacs)

# Переделаем полученные свойства в array 
train_data_features = train_data_features.toarray()

Фигачим Bag of words...

CPU times: user 15.7 s, sys: 3.23 s, total: 18.9 s
Wall time: 19.1 s


In [17]:
train_data_features.shape

(200000, 5000)

In [18]:
# Посмотрим на получившийся словарь 
vocab = vectorizer.get_feature_names()
print (vocab)

['abap', 'abilit', 'abl', 'about', 'access', 'according', 'account', 'accounts', 'achiev', 'across', 'activ', 'activities', 'ad', 'additiona', 'administration', 'adob', 'advanced', 'advantag', 'advertising', 'adwords', 'after', 'agil', 'ajax', 'algorithms', 'all', 'als', 'always', 'amazon', 'amocrm', 'among', 'an', 'analysis', 'analyst', 'analytica', 'analytics', 'analyz', 'and', 'android', 'angular', 'angularjs', 'annua', 'ansibl', 'ap', 'apach', 'apis', 'app', 'appl', 'application', 'applications', 'approach', 'approaches', 'appropriat', 'apps', 'ar', 'architect', 'architectur', 'are', 'areas', 'arm', 'around', 'art', 'as', 'asp', 'assigned', 'assist', 'assuranc', 'asterisk', 'at', 'atlassia', 'atmospher', 'attention', 'attitud', 'aud', 'autocad', 'automated', 'automation', 'availabilit', 'availabl', 'aws', 'azur', 'bachelor', 'back', 'backbon', 'backend', 'background', 'backup', 'bas', 'based', 'bash', 'basic', 'basis', 'be', 'beaut', 'becom', 'behalf', 'being', 'belarus', 'benefits

In [19]:
import numpy as np

# Просуммируем счетчики
dist = np.sum(train_data_features, axis=0)

# Посмотрим сколько раз каждое слово встречается в train_df

for tag, count in zip(vocab, dist):
    print (count, tag)

435 abap
3514 abilit
1024 abl
1732 about
512 access
654 according
765 account
317 accounts
374 achiev
917 across
1107 activ
1417 activities
1465 ad
428 additiona
602 administration
2434 adob
990 advanced
698 advantag
323 advertising
1832 adwords
342 after
2794 agil
1249 ajax
298 algorithms
3024 all
784 als
316 always
523 amazon
294 amocrm
416 among
5145 an
1556 analysis
313 analyst
884 analytica
2446 analytics
507 analyz
74126 and
4295 android
1970 angular
1024 angularjs
376 annua
569 ansibl
3961 ap
1299 apach
339 apis
661 app
1437 appl
1837 application
1815 applications
593 approach
282 approaches
319 appropriat
388 apps
8713 ar
508 architect
1307 architectur
571 are
490 areas
327 arm
764 around
390 art
8339 as
1734 asp
292 assigned
340 assist
335 assuranc
398 asterisk
3381 at
310 atlassia
425 atmospher
331 attention
289 attitud
330 aud
2301 autocad
521 automated
1680 automation
310 availabilit
370 availabl
916 aws
542 azur
470 bachelor
1097 back
304 backbon
1360 backend
551 backgroun

69556 активн
1486 актуализац
5287 актуальн
10281 акц
326 акцион
430 ал
2749 алгоритм
479 алкогол
1360 алкогольн
445 алмат
283 алтайск
355 альн
403 альтернативн
379 альф
278 алюминиев
1232 амбиц
8973 амбициозн
297 америк
541 американск
2005 амортизац
447 ана
37496 анализ
552 анализир
3126 анализирова
9307 аналитик
11175 аналитическ
1151 аналог
13011 аналогичн
379 аналогов
561 англ
20459 английск
524 анимац
1083 анкет
715 антивирусн
2812 аппарат
1834 аппаратн
869 аппаратур
824 аптек
353 аптечн
700 аргументирова
2545 аренд
409 арендатор
268 арм
565 арматур
271 ароматн
343 арт
298 архив
1213 архитектор
5424 архитектур
1263 архитектурн
1228 ас
270 асинхрон
630 аспект
359 ассистент
17072 ассортимент
2165 ассортиментн
451 асутп
4329 атмосфер
686 атс
1986 аттестац
2616 ауд
2164 аудит
3172 аудитор
676 аукцион
462 аутсорсинг
310 аутсорсингов
327 аша
767 аэропорт
859 баг
635 бажан
48014 баз
8735 базов
327 бакалейн
567 баланс
294 балаших
358 балтик
10322 банк
4119 банковск
1270 баннер
529 бар
288 

340 коп
1033 копирайтер
898 копирайтинг
305 копировальн
1380 копирован
341 кор
333 кораблик
1124 корм
959 кормлен
301 кормов
1150 коротк
273 корп
697 корпоратив
115864 корпоративн
1055 корпорац
354 корпус
970 корпусн
534 корректир
3084 корректировк
1905 корректн
312 коррекц
389 корреспонденц
3435 косметик
1153 косметическ
300 косметолог
562 котельн
346 котл
34876 котор
316 коучинг
6497 коф
412 кофейн
354 кофемашин
302 коэффициент
515 кп
574 кпк
1983 кра
527 крайн
268 кран
1955 красив
280 краск
994 красн
1002 краснодар
598 краснодарск
775 красноярск
2511 красот
536 кратк
737 краткосрочн
373 кратчайш
453 креатив
5676 креативн
522 кред
766 кредит
953 кредитн
2123 кредитован
438 крепк
493 кресл
312 криптовалют
851 критер
424 критическ
307 кровельн
988 кром
1009 кросс
559 кроссбраузерн
553 крс
702 круг
616 кругл
322 круглосуточн
953 кругозор
41638 крупн
6236 крут
474 крым
372 кс
280 ку
312 куб
352 кулинарн
283 культов
6726 культур
578 культурн
538 куп
597 купл
384 куратор
857 курирован
502 

639 принос
1081 принтер
14749 принцип
719 принципиальн
2147 приня
8840 принят
608 приобрест
710 приобрета
2312 приобретен
2438 приоритет
711 приоритетн
356 природ
314 природн
342 природоохра
1352 присоедин
3175 присоединя
838 приспособлен
1921 присутств
1389 присыла
5034 приход
3231 причин
378 пришл
2482 приятн
1320 проактивн
692 проб
3468 пробл
1666 проблем
480 проблемн
1750 провайдер
52136 проведен
294 проверен
31161 проверк
1115 проверок
5207 проверя
361 провест
10466 провод
1406 проводим
277 проводн
899 прогноз
1307 прогнозирован
37384 программ
294 программирова
9454 программирован
5496 программист
14305 программн
1520 прогрессивн
355 прод
2872 прода
427 продав
8580 продава
5545 продавец
12963 продавц
184434 продаж
377 продажник
1103 продвига
16630 продвижен
2710 продвинут
2255 продела
362 продлен
1082 продовольствен
730 продолжа
607 продолжительн
40506 продукт
1132 продуктивн
4100 продуктов
60510 продукц
4656 проезд
63576 проект
561 проектирова
12193 проектирован
563 проектировщик

282 шеф
459 шин
5011 широк
513 шкаф
2006 школ
326 шмр
536 шо
278 шоколад
1872 шосс
6150 штат
676 штатн
312 штрих
429 що
341 экк
744 эколог
1285 экологическ
535 эконом
2461 экономик
5645 экономическ
1783 эксклюзивн
356 экскурс
383 эксперимент
495 экспериментальн
3442 эксперт
2401 экспертиз
1209 экспертн
9372 эксплуатац
1295 эксплуатацион
399 экспорт
493 экспресс
1090 эл
322 элегантн
348 электр
824 электрик
2543 электрическ
916 электробезопасн
534 электроинструмент
351 электромонтажн
8786 электрон
1888 электроник
1488 электрооборудован
471 электроснабжен
997 электротехник
2699 электротехническ
419 электроустановок
736 электроэнерг
441 электроэнергетик
2074 элемент
496 элитн
350 эльдорад
519 эмоц
480 эмоциональн
1521 энерг
1416 энергетик
2062 энергетическ
7816 энергичн
596 энтузиазм
378 энтузиаст
589 эскиз
344 эскизн
27372 эт
744 этаж
6654 этап
2207 этик
619 этикет
293 эффект
23577 эффективн
2992 ювелирн
915 юг
617 южн
614 юзабилит
786 юмор
304 юн
299 юр
4418 юридическ
451 юрист
15979 явл

## Шаг 3. Решам задачу через модель Random Forest

In [20]:
#Делаем трейн тест сплит

from sklearn.cross_validation import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

X_train, X_test, y_train, y_test = train_test_split(train_data_features, train_df['target'], test_size=0.33, random_state=42)
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)


/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


X_train  (134000, 5000)
X_test  (66000, 5000)
y_train  (134000,)
y_test  (66000,)


In [21]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Пока делаем 100 деревьев, но в следующих вариантах домашки попробую сделать 50
forest = RandomForestClassifier(n_estimators = 100) 

# В версии с 5 000 фич обучение заняло 12 минут 41 секунду, старт в 23-25

forest = forest.fit( X_train, y_train )
print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 9min 59s, sys: 7.76 s, total: 10min 7s
Wall time: 10min 11s


In [22]:
predict = forest.predict(X_test)

In [23]:
# прошлый результаты были 0.96 0.96 0.96

print('#'*20,'>>> Random Trees <<<','#'*20,'\n',classification_report(y_test, predict),'\n')

#################### >>> Random Trees <<< #################### 
              precision    recall  f1-score   support

          0       0.97      0.96      0.96     34984
          1       0.95      0.97      0.96     31016

avg / total       0.96      0.96      0.96     66000
 



# Шаг 4. Готовим файл к оправке

In [24]:
%%time

print ('Обучаем Random Forest...\n')
from sklearn.ensemble import RandomForestClassifier

# Делаем random forest на 100 деревьев
forest = RandomForestClassifier(n_estimators = 100) 

# В версии с 5 000 фич обучение заняло 22 минут 12 секунд, старт в 23-37
forest = forest.fit( train_data_features, train_df['target'] )

print ('Саш, я обучилась!')

Обучаем Random Forest...

Саш, я обучилась!
CPU times: user 18min 13s, sys: 10.6 s, total: 18min 23s
Wall time: 18min 29s


In [25]:
test_df.shape

(170179, 3)

In [26]:
num_test_vacs = len(test_df['description'])

In [32]:
%%time
clean_test_vacs = []
print ('Очищаем и собираем нормализованные описания из test_df...\n')
for i in range(0,num_test_vacs):
    if( (i+1) % 10000 == 0 ):
        print ('Описание {} из {} в обработке...'.format( i+1, num_test_vacs)) 
    clean_test_vacs.append( vac_to_words( test_df['description'][i] , True) )
print('Я закончила!!!')

Очищаем и собираем нормализованные описания из test_df...

Описание 10000 из 170179 в обработке...
Описание 20000 из 170179 в обработке...
Описание 30000 из 170179 в обработке...
Описание 40000 из 170179 в обработке...
Описание 50000 из 170179 в обработке...
Описание 60000 из 170179 в обработке...
Описание 70000 из 170179 в обработке...
Описание 80000 из 170179 в обработке...
Описание 90000 из 170179 в обработке...
Описание 100000 из 170179 в обработке...
Описание 110000 из 170179 в обработке...
Описание 120000 из 170179 в обработке...
Описание 130000 из 170179 в обработке...
Описание 140000 из 170179 в обработке...
Описание 150000 из 170179 в обработке...
Описание 160000 из 170179 в обработке...
Описание 170000 из 170179 в обработке...
Я закончила!!!
CPU times: user 22min 48s, sys: 7.73 s, total: 22min 56s
Wall time: 22min 58s


In [33]:
%%time
# Фигачим bag of words по тестовым данным

test_data_features = vectorizer.transform(clean_test_vacs)
test_data_features = test_data_features.toarray()

CPU times: user 13.4 s, sys: 2.09 s, total: 15.5 s
Wall time: 15.6 s


In [34]:
%%time
# Через Random Forest делаем предсказание
result_test = forest.predict(test_data_features)

CPU times: user 22.2 s, sys: 3.2 s, total: 25.4 s
Wall time: 16.4 s


In [35]:
# Копируем результат в датафрейм с полями id и target 
# предыдущий реузльтат был таким http://take.ms/cVRK4
output = pd.DataFrame( data={'id':test_df['id'], 'target':result_test} )
output.head()

,id,target
0,200000,1
1,200001,1
2,200002,1
3,200003,1
4,200004,0


In [37]:
output.to_csv( "Random_Forest_5000stem.csv", sep=',', encoding='utf-8',index=False )